In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [ ]:
# For computation
import numpy as np
import math
import scipy.stats as stats
import matplotlib.pyplot as plt
from shapely import geometry
from scipy.spatial.distance import directed_hausdorff

In [ ]:
DATASTORE_URL = "http://cardshark.cs.berkeley.edu"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sdunp = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "unimodal_trip_car_bike_mtv_la")

In [ ]:
import importlib
importlib.reload(ezpv)

In [ ]:
pvunp = eipv.PhoneView(sdunp)

In [ ]:
polygons = []
trips = pvunp.spec_details.curr_spec['evaluation_trips']

for trip in trips:
    for leg in trip['legs']:
        if 'loc' in leg and leg['loc']['geometry']['type'] == 'Polygon':
            polygons.append(geometry.Polygon(leg['loc']['geometry']['coordinates'][0]))
        if 'end_loc' in leg and leg['end_loc']['geometry']['type'] == 'Polygon':
            polygons.append(geometry.Polygon(leg['end_loc']['geometry']['coordinates'][0]))
        if 'start_loc' in leg and leg['start_loc']['geometry']['type'] == 'Polygon':
            polygons.append(geometry.Polygon(leg['start_loc']['geometry']['coordinates'][0]))

In [ ]:
def clean(arr):
    """
    Latitude and Longitude defined as np array with shape (n,)
    Reshape to (n,1)
    """
    return arr.values.reshape((arr.shape[0], 1))

def is_point_in_polygon(point):
    """
    Utility function to check if given Shapely point
    is contained within a Shapely polygon
    """
    for polygon in polygons:
        if polygon.contains(point):
            return True
    return False

def get_trajectory(df, include_timestamps=True):
    """ 
    Returns np array combining latitudes, longitudes
    and timestamps (if asked for). Only returns those
    points that are not contained within Parking Polygons.
    """
    latitudes = clean(df['latitude'])
    longitudes = clean(df['longitude'])
    if include_timestamps:
        timestamps = clean(df['ts'])
    
    coordinates = np.concatenate((longitudes, latitudes), axis=1).tolist()
    
    data_points = []
    
    for (i, coordinate) in enumerate(coordinates):
        point = geometry.Point(coordinate[0], coordinate[1])
        
        if not is_point_in_polygon(point):
            if include_timestamps:
                coordinate.append(timestamps[i][0])
        data_points.append(coordinate)
    
    return data_points

def get_gt_points(pvunp, phone_os, phone_label, trip_index, section_index):
    """
    Get lat-long corrdinates in ground truth
    """
    data = pvunp.spec_details.curr_spec['evaluation_trips'][trip_index]['legs'][section_index]
    if 'route_coords' in data:
        return data['route_coords']['geometry']['coordinates']
    return []

def get_measured_points(df):
    """
    Get lat-long coordinates recorded from phone
    """
    return get_trajectory(df, include_timestamps=False)

def compute_hausdorff_similarity(base_df, test_df):
    """
    Compute Hausfdorff distance between lat-long-timestamp
    triplets of different measurements.
    """
    base_trajectory = get_trajectory(base_df)
    test_trajectory = get_trajectory(test_df)
    return directed_hausdorff(base_trajectory, test_trajectory)[0]

def get_projections(line, measured_points):
    projections = []
    for p in measured_points:
        point = geometry.Point(p[0], p[1])
        projections.append(point.distance(line))
    return projections

def plot_hist(points):
    plt.hist(points, bins=10)
    plt.show()
    
def convert_to_xy(points):
    xy_points = []
    R = 6371 # Radius of the earth in km
    
    for point in points:
        lon = point[0]
        lat = point[1]
        
        theta = math.pi/2 - math.radians(lat) 
        phi = math.radians(lon)
        
        x = R * math.sin(theta) * math.cos(phi)
        y = R * math.sin(theta) * math.sin(phi)
        xy_points.append([x, y])
        
    return xy_points

In [ ]:
spatial_errors = {}


"""
    Spatial errors is a data structure that stores the individual error points (projections) as well as the 
    RMS error for every phone for every one. It is structured as: 
        spatial_errors
        -- ios
        ----ucb-sdb-ios-1
        ------total_error[0] For the different trips
        ------total_error[1]
        .
        .
        ------errors[0] For the different trips
        ------errors[1]
        .
        .
        ----ucb-sdb-ios-2
        .
        .
        -- android
        ----ucb-sdb-android-1
        
        # Errors are all reported in miles
        
"""
for phone_os, phone_map in pvunp.map().items():
    spatial_errors[phone_os] = {}
    
    for phone_label, phone_detail_map in phone_map.items():
        spatial_errors[phone_os][phone_label] = {}
        spatial_errors[phone_os][phone_label]['total_error'] = []
        spatial_errors[phone_os][phone_label]['errors'] = []
        
        for (r_idx, r) in enumerate(phone_detail_map["evaluation_ranges"]):
            run_mean_error = 0.0
            run_errors = []
            
            for (tr_idx, tr) in enumerate(r["evaluation_trip_ranges"]):
                trip_mean_error = 0.0
                trip_errors = []
                    
                for (sr_idx, sr) in enumerate(tr["evaluation_section_ranges"]):
                    gt_points = get_gt_points(pvunp, phone_os, phone_label, tr_idx, sr_idx)
                    
                    if len(sr['location_df']) == 0:
                        continue
                        
                    measured_points = get_measured_points(sr['location_df'])
                    line = geometry.LineString(convert_to_xy(gt_points))
                    projections = get_projections(line, convert_to_xy(measured_points))
                    section_error = sum(projections)
                    section_observations = len(projections)
                    section_mean_error = section_error/section_observations
                    trip_mean_error += section_mean_error
                    trip_errors += projections
                    
                run_mean_error += trip_mean_error
                run_errors += trip_errors
                
            spatial_errors[phone_os][phone_label]['total_error'].append(run_mean_error)
            spatial_errors[phone_os][phone_label]['errors'].append(run_errors)

In [ ]:
plot_hist(spatial_errors['ios']['ucb-sdb-ios-1']['errors'][0])

In [ ]:
plot_hist(spatial_errors['ios']['ucb-sdb-ios-1']['errors'][1])

In [ ]:
plot_hist(spatial_errors['ios']['ucb-sdb-ios-1']['errors'][2])

In [ ]:
plot_hist(spatial_errors['ios']['ucb-sdb-ios-2']['errors'][0])

In [ ]:
plot_hist(spatial_errors['ios']['ucb-sdb-ios-2']['errors'][1])

In [ ]:
plot_hist(spatial_errors['ios']['ucb-sdb-ios-2']['errors'][2])

In [ ]:
# Total error of ucb-sdb-ios-1 for each trip 
print(spatial_errors['ios']['ucb-sdb-ios-1']['total_error'])

In [ ]:
print(spatial_errors['ios']['ucb-sdb-ios-2']['total_error'])

In [ ]:
print(spatial_errors['ios']['ucb-sdb-ios-3']['total_error'])

In [ ]:
print(spatial_errors['android']['ucb-sdb-android-1']['total_error'])

In [ ]:
print(spatial_errors['android']['ucb-sdb-android-2']['total_error'])

In [ ]:
print(spatial_errors['android']['ucb-sdb-android-3']['total_error'])